<a href="https://colab.research.google.com/github/AnkitaChakraborty1/Capstone_Adversarial_Robustness/blob/main/PruthiRecipes_AllenNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##pip installs

In [1]:
pip install transformers

     |████████████████████████████████| 3.8 MB 4.9 MB/s 
     |████████████████████████████████| 67 kB 5.2 MB/s 
     |████████████████████████████████| 596 kB 43.2 MB/s 
     |████████████████████████████████| 6.5 MB 45.6 MB/s 
     |████████████████████████████████| 895 kB 41.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
!pip3 install textattack[tensorflow]

     |████████████████████████████████| 373 kB 5.1 MB/s 
     |████████████████████████████████| 322 kB 46.0 MB/s 
     |████████████████████████████████| 101 kB 6.9 MB/s 
     |████████████████████████████████| 69 kB 7.2 MB/s 
     |████████████████████████████████| 769 kB 33.0 MB/s 
     |████████████████████████████████| 325 kB 61.5 MB/s 
     |████████████████████████████████| 60 kB 8.0 MB/s 
     |████████████████████████████████| 125 kB 74.0 MB/s 
     |████████████████████████████████| 4.9 MB 41.6 MB/s 
     |████████████████████████████████| 462 kB 54.5 MB/s 
     |████████████████████████████████| 454.3 MB 15 kB/s 
     |████████████████████████████████| 4.0 MB 51.9 MB/s 
     |████████████████████████████████| 1.2 MB 38.2 MB/s 
     |████████████████████████████████| 132 kB 64.6 MB/s 
     |████████████████████████████████| 14.8 MB 28.6 MB/s 
     |████████████████████████████████| 4.9 MB 52.0 MB/s 
     |████████████████████████████████| 4.9 MB 41.8 MB/s 
     |█████████████

In [1]:
pip install --upgrade numpy

     |████████████████████████████████| 15.7 MB 5.1 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.5.0 requires numpy~=1.19.2, but you have numpy 1.21.5 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [2]:
pip install folium==0.2.1

     |████████████████████████████████| 69 kB 3.5 MB/s 
  Created wheel for folium: filename=folium-0.2.1-py3-none-any.whl size=79808 sha256=8a395bf11654e8e7d490c40461299ba00ea3e927e14a3e0d72b7428f20286bac
  Stored in directory: /root/.cache/pip/wheels/9a/f0/3a/3f79a6914ff5affaf50cabad60c9f4d565283283c97f0bdccf
Successfully built folium
  Attempting uninstall: folium
    Found existing installation: folium 0.8.3
    Uninstalling folium-0.8.3:
      Successfully uninstalled folium-0.8.3


## Imports

In [1]:
from textattack import AttackArgs, Attacker
from textattack.datasets import HuggingFaceDataset
from textattack.models.wrappers import ModelWrapper
from textattack import Attack
from textattack.constraints.pre_transformation import (
        InputColumnModification,
        MaxModificationRate,
        RepeatModification,
        StopwordModification,
        MaxWordIndexModification
                )
from textattack.constraints.semantics import WordEmbeddingDistance
from textattack.goal_functions import UntargetedClassification,InputReduction,TargetedClassification
from textattack.search_methods import GreedyWordSwapWIR
from textattack.transformations import (
   
    WordSwapNeighboringCharacterSwap,
    WordSwapQWERTY,
    WordSwapWordNet,
    WordSwapRandomCharacterDeletion,
    WordSwapRandomCharacterInsertion,
    WordSwapRandomCharacterSubstitution,
    WordSwapHomoglyphSwap,
    WordSwapEmbedding,
    WordSwapHowNet,
    WordSwapInflections,
    WordDeletion,
    WordSwapGradientBased,
    WordSwapExtend,
    WordSwapContract,
    WordSwapChangeName,
    WordSwapChangeNumber,
    WordSwapChangeLocation,
    WordSwapRandomCharacterSubstitution,
    WordSwapMaskedLM,
    WordInsertionMaskedLM,
    WordMergeMaskedLM,
)
from textattack.constraints.overlap import LevenshteinEditDistance
#from textattack.attack_recipe import AttackRecipe

from abc import ABC, abstractmethod
from tqdm import tqdm # tqdm provides us a nice progress bar.
from textattack.loggers import CSVLogger # tracks a dataframe for us.
from IPython.core.display import display, HTML

textattack: Downloading https://textattack.s3.amazonaws.com/word_embeddings/paragramcf.
100%|██████████| 481M/481M [00:10<00:00, 44.5MB/s]
textattack: Unzipping file /root/.cache/textattack/tmpd9ff82v5.zip to /root/.cache/textattack/word_embeddings/paragramcf.
textattack: Successfully saved word_embeddings/paragramcf to cache.


Creating the AlbertBase Model

In [3]:
from textattack.datasets import HuggingFaceDataset
from textattack.attacker import Attacker
from textattack import AttackArgs, Attacker
from textattack.datasets import HuggingFaceDataset
from textattack.goal_functions import TargetedClassification
import transformers
from textattack.models.wrappers import HuggingFaceModelWrapper
import torch

model = transformers.AutoModelForSequenceClassification.from_pretrained("textattack/albert-base-v2-imdb")
tokenizer = transformers.AutoTokenizer.from_pretrained("textattack/albert-base-v2-imdb")
# We wrap the model so it can be used by textattack
model_wrapper = HuggingFaceModelWrapper(model, tokenizer)

dataset = HuggingFaceDataset("rotten_tomatoes", None, "test", shuffle=True)




Using custom data configuration default
Reusing dataset rotten_tomatoes_movie_review (/root/.cache/huggingface/datasets/rotten_tomatoes_movie_review/default/1.0.0/9198dbc50858df8bdb0d5f18ccaf33125800af96ad8434bc8b829918c987ee8a)
textattack: Loading datasets dataset rotten_tomatoes, split test.


##Custom Recipes

In [5]:
from textattack import Attack
from textattack.constraints.overlap import MaxWordsPerturbed
from textattack.constraints.pre_transformation import (
    MinWordLength,
    RepeatModification,
    StopwordModification,
)
from textattack.goal_functions import UntargetedClassification
from textattack.search_methods import GreedySearch
from textattack.transformations import (
    CompositeTransformation,
    WordSwapNeighboringCharacterSwap,
    WordSwapQWERTY,
    WordSwapRandomCharacterDeletion,
    WordSwapRandomCharacterInsertion,
    WordSwapEmbedding,
)

class Recipe1(Attack,ABC):
    @staticmethod
    def build(model_wrapper):
         # a combination of 4 different character-based transforms
        # ignore the first and last letter of each word, as in the paper
        transformation = CompositeTransformation(
            [
                WordSwapNeighboringCharacterSwap(
                    random_one=False, skip_first_char=True, skip_last_char=True
                ),
                WordSwapRandomCharacterDeletion(
                    random_one=False, skip_first_char=True, skip_last_char=True
                ),
                WordSwapRandomCharacterInsertion(
                    random_one=False, skip_first_char=True, skip_last_char=True
                ),
                WordSwapQWERTY(
                    random_one=False, skip_first_char=True, skip_last_char=True
                ),
            ]
        )
        # only edit words of length >= 4, edit max_num_word_swaps words.
        # note that we also are not editing the same word twice, so
        # max_num_word_swaps is really the max number of character
        # changes that can be made. The paper looks at 1 and 2 char attacks.
        constraints = [
            MinWordLength(min_length=4),
            StopwordModification(),
            MaxWordsPerturbed(max_num_words=2),
            RepeatModification(),
        ]
        # untargeted attack
        goal_function = UntargetedClassification(model_wrapper)
        search_method = GreedySearch()
        return Attack(goal_function, constraints, transformation, search_method)


In [6]:

attack = Recipe1.build(model_wrapper)
attack_args = AttackArgs(num_examples=100,)
attacker = Attacker(attack, dataset,attack_args)
attack_results = attacker.attack_dataset()

textattack: Unknown if model of class <class 'transformers.models.albert.modeling_albert.AlbertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
  0%|          | 0/100 [00:00<?, ?it/s]

Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  CompositeTransformation(
    (0): WordSwapNeighboringCharacterSwap(
        (random_one):  False
      )
    (1): WordSwapRandomCharacterDeletion(
        (random_one):  False
      )
    (2): WordSwapRandomCharacterInsertion(
        (random_one):  False
      )
    (3): WordSwapQWERTY
    )
  (constraints): 
    (0): MaxWordsPerturbed(
        (max_num_words):  2
        (compare_against_original):  True
      )
    (1): MinWordLength
    (2): StopwordModification
    (3): RepeatModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:   1%|          | 1/100 [00:20<33:52, 20.53s/it]

--------------------------------------------- Result 1 ---------------------------------------------
[[Positive (100%)]] --> [[Negative (99%)]]

lovingly photographed in the manner of a golden book sprung to life , stuart little 2 [[manages]] [[sweetness]] largely without stickiness .

lovingly photographed in the manner of a golden book sprung to life , stuart little 2 [[managss]] [[sweetnfss]] largely without stickiness .




[Succeeded / Failed / Skipped / Total] 2 / 0 / 1 / 3:   3%|▎         | 3/100 [00:27<14:41,  9.09s/it]

...

--------------------------------------------- Result 98 ---------------------------------------------
[[Positive (95%)]] --> [[Negative (81%)]]

[[connoisseurs]] of chinese film will be pleased to discover that tian's [[meticulous]] talent has not withered during his enforced hiatus .

[[connoiqseurs]] of chinese film will be pleased to discover that tian's [[msticulous]] talent has not withered during his enforced hiatus .




[Succeeded / Failed / Skipped / Total] 82 / 7 / 10 / 99:  99%|█████████▉| 99/100 [14:58<00:09,  9.08s/it]

--------------------------------------------- Result 99 ---------------------------------------------
[[Positive (99%)]] --> [[Negative (90%)]]

if you can push on through the slow spots , you'll be rewarded with some [[fine]] acting .

if you can push on through the slow spots , you'll be rewarded with some [[fjne]] acting .




[Succeeded / Failed / Skipped / Total] 83 / 7 / 10 / 100: 100%|██████████| 100/100 [15:07<00:00,  9.07s/it]

--------------------------------------------- Result 100 ---------------------------------------------
[[Positive (96%)]] --> [[Negative (97%)]]

an [[unusually]] dry-eyed , even analytical approach to material that is generally played for maximum moisture .

an [[unusuxlly]] dry-eyed , even analytical approach to material that is generally played for maximum moisture .



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 83     |
| Number of failed attacks:     | 7      |
| Number of skipped attacks:    | 10     |
| Original accuracy:            | 90.0%  |
| Accuracy under attack:        | 7.0%   |
| Attack success rate:          | 92.22% |
| Average perturbed word %:     | 9.78%  |
| Average num. words per input: | 18.45  |
| Avg num queries:              | 499.17 |
+-------------------------------+--------+


In [5]:

class Recipe2(Attack,ABC):
    @staticmethod
    def build(model_wrapper):
         # a combination of 4 different character-based transforms
        # ignore the first and last letter of each word, as in the paper
        transformation = CompositeTransformation(
            [
                WordSwapNeighboringCharacterSwap(
                    random_one=False, skip_first_char=True, skip_last_char=True
                ),
                WordSwapRandomCharacterDeletion(
                    random_one=False, skip_first_char=True, skip_last_char=True
                ),
                WordSwapRandomCharacterInsertion(
                    random_one=False, skip_first_char=True, skip_last_char=True
                ),
                WordSwapQWERTY(
                    random_one=False, skip_first_char=True, skip_last_char=True
                ),
            ]
        )
        constraints = [
            MinWordLength(min_length=4),
            StopwordModification(),
            MaxWordsPerturbed(max_num_words=2),
            RepeatModification(),
        ]
        # untargeted attack
        goal_function = UntargetedClassification(model_wrapper)
        search_method = GreedyWordSwapWIR(wir_method="delete")
        return Attack(goal_function, constraints, transformation, search_method)


In [6]:
attack = Recipe2.build(model_wrapper)
attack_args = AttackArgs(num_examples=100,)
attacker = Attacker(attack, dataset,attack_args)
attack_results = attacker.attack_dataset()

textattack: Unknown if model of class <class 'transformers.models.albert.modeling_albert.AlbertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
  0%|          | 0/100 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  CompositeTransformation(
    (0): WordSwapNeighboringCharacterSwap(
        (random_one):  False
      )
    (1): WordSwapRandomCharacterDeletion(
        (random_one):  False
      )
    (2): WordSwapRandomCharacterInsertion(
        (random_one):  False
      )
    (3): WordSwapQWERTY
    )
  (constraints): 
    (0): MaxWordsPerturbed(
        (max_num_words):  2
        (compare_against_original):  True
      )
    (1): MinWordLength
    (2): StopwordModification
    (3): RepeatModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:   1%|          | 1/100 [00:04<06:36,  4.00s/it]

--------------------------------------------- Result 1 ---------------------------------------------
[[Positive (100%)]] --> [[Negative (98%)]]

lovingly photographed in the manner of a golden book sprung to life , stuart little 2 [[manages]] [[sweetness]] largely without stickiness .

lovingly photographed in the manner of a golden book sprung to life , stuart little 2 [[manabes]] [[sweetnfss]] largely without stickiness .




[Succeeded / Failed / Skipped / Total] 2 / 0 / 1 / 3:   3%|▎         | 3/100 [00:07<04:09,  2.57s/it]

...

--------------------------------------------- Result 98 ---------------------------------------------
[[Positive (95%)]] --> [[[FAILED]]]

connoisseurs of chinese film will be pleased to discover that tian's meticulous talent has not withered during his enforced hiatus .




[Succeeded / Failed / Skipped / Total] 64 / 25 / 10 / 99:  99%|█████████▉| 99/100 [04:38<00:02,  2.81s/it]

--------------------------------------------- Result 99 ---------------------------------------------
[[Positive (99%)]] --> [[Negative (90%)]]

if you can push on through the slow spots , you'll be rewarded with some [[fine]] acting .

if you can push on through the slow spots , you'll be rewarded with some [[fjne]] acting .




[Succeeded / Failed / Skipped / Total] 65 / 25 / 10 / 100: 100%|██████████| 100/100 [04:41<00:00,  2.81s/it]

--------------------------------------------- Result 100 ---------------------------------------------
[[Positive (96%)]] --> [[Negative (95%)]]

an unusually dry-eyed , even [[analytical]] approach to material that is generally played for maximum moisture .

an unusually dry-eyed , even [[abalytical]] approach to material that is generally played for maximum moisture .



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 65     |
| Number of failed attacks:     | 25     |
| Number of skipped attacks:    | 10     |
| Original accuracy:            | 90.0%  |
| Accuracy under attack:        | 25.0%  |
| Attack success rate:          | 72.22% |
| Average perturbed word %:     | 10.65% |
| Average num. words per input: | 18.45  |
| Avg num queries:              | 84.3   |
+-------------------------------+--------+


In [7]:
class Recipe3(Attack,ABC):
    @staticmethod
    def build(model_wrapper):
         # a combination of 4 different character-based transforms
        # ignore the first and last letter of each word, as in the paper
        transformation = CompositeTransformation(
            [
                WordSwapNeighboringCharacterSwap(
                    random_one=False, skip_first_char=True, skip_last_char=True
                ),
                WordSwapRandomCharacterDeletion(
                    random_one=False, skip_first_char=True, skip_last_char=True
                ),
                WordSwapRandomCharacterInsertion(
                    random_one=False, skip_first_char=True, skip_last_char=True
                ),
                WordSwapQWERTY(
                    random_one=False, skip_first_char=True, skip_last_char=True
                ),
            ]
        )
        constraints = [
            MinWordLength(min_length=6),
            StopwordModification(),
            MaxWordsPerturbed(max_num_words=2),
            RepeatModification(),
        ]
        # untargeted attack
        goal_function = UntargetedClassification(model_wrapper)
        search_method = GreedyWordSwapWIR(wir_method="delete")
        return Attack(goal_function, constraints, transformation, search_method)

attack = Recipe3.build(model_wrapper)
attack_args = AttackArgs(num_examples=100,)
attacker = Attacker(attack, dataset,attack_args)
attack_results = attacker.attack_dataset()

textattack: Unknown if model of class <class 'transformers.models.albert.modeling_albert.AlbertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
  0%|          | 0/100 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  CompositeTransformation(
    (0): WordSwapNeighboringCharacterSwap(
        (random_one):  False
      )
    (1): WordSwapRandomCharacterDeletion(
        (random_one):  False
      )
    (2): WordSwapRandomCharacterInsertion(
        (random_one):  False
      )
    (3): WordSwapQWERTY
    )
  (constraints): 
    (0): MaxWordsPerturbed(
        (max_num_words):  2
        (compare_against_original):  True
      )
    (1): MinWordLength
    (2): StopwordModification
    (3): RepeatModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:   1%|          | 1/100 [00:04<07:12,  4.37s/it]

--------------------------------------------- Result 1 ---------------------------------------------
[[Positive (100%)]] --> [[Negative (98%)]]

lovingly photographed in the manner of a golden book sprung to life , stuart little 2 [[manages]] [[sweetness]] largely without stickiness .

lovingly photographed in the manner of a golden book sprung to life , stuart little 2 [[manabes]] [[sweetnfss]] largely without stickiness .




[Succeeded / Failed / Skipped / Total] 2 / 0 / 1 / 3:   3%|▎         | 3/100 [00:08<04:34,  2.83s/it]

...

--------------------------------------------- Result 98 ---------------------------------------------
[[Positive (95%)]] --> [[[FAILED]]]

connoisseurs of chinese film will be pleased to discover that tian's meticulous talent has not withered during his enforced hiatus .




[Succeeded / Failed / Skipped / Total] 58 / 31 / 10 / 99:  99%|█████████▉| 99/100 [05:35<00:03,  3.39s/it]

--------------------------------------------- Result 99 ---------------------------------------------
[[Positive (99%)]] --> [[Negative (76%)]]

if you can push on through the slow spots , you'll be [[rewarded]] with some fine acting .

if you can push on through the slow spots , you'll be [[rewared]] with some fine acting .




[Succeeded / Failed / Skipped / Total] 59 / 31 / 10 / 100: 100%|██████████| 100/100 [05:38<00:00,  3.38s/it]

--------------------------------------------- Result 100 ---------------------------------------------
[[Positive (96%)]] --> [[Negative (95%)]]

an unusually dry-eyed , even [[analytical]] approach to material that is generally played for maximum moisture .

an unusually dry-eyed , even [[abalytical]] approach to material that is generally played for maximum moisture .



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 59     |
| Number of failed attacks:     | 31     |
| Number of skipped attacks:    | 10     |
| Original accuracy:            | 90.0%  |
| Accuracy under attack:        | 31.0%  |
| Attack success rate:          | 65.56% |
| Average perturbed word %:     | 11.09% |
| Average num. words per input: | 18.45  |
| Avg num queries:              | 97.39  |
+-------------------------------+--------+


In [8]:
class Recipe4(Attack,ABC):
    @staticmethod
    def build(model_wrapper):
         # a combination of 4 different character-based transforms
        # ignore the first and last letter of each word, as in the paper
        transformation = CompositeTransformation(
            [
                WordSwapNeighboringCharacterSwap(
                    random_one=False, skip_first_char=True, skip_last_char=True
                ),
                WordSwapRandomCharacterDeletion(
                    random_one=False, skip_first_char=True, skip_last_char=True
                ),
                WordSwapRandomCharacterInsertion(
                    random_one=False, skip_first_char=True, skip_last_char=True
                ),
                WordSwapQWERTY(
                    random_one=False, skip_first_char=True, skip_last_char=True
                ),
                WordSwapEmbedding(max_candidates=5),
            ]
        )
        constraints = [
            MinWordLength(min_length=6),
            StopwordModification(),
            MaxWordsPerturbed(max_num_words=4),
            RepeatModification(),
        ]
        # untargeted attack
        goal_function = UntargetedClassification(model_wrapper)
        search_method = GreedyWordSwapWIR(wir_method="delete")
        return Attack(goal_function, constraints, transformation, search_method)

attack = Recipe4.build(model_wrapper)
attack_args = AttackArgs(num_examples=100,)
attacker = Attacker(attack, dataset,attack_args)
attack_results = attacker.attack_dataset()

textattack: Unknown if model of class <class 'transformers.models.albert.modeling_albert.AlbertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
  0%|          | 0/100 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  CompositeTransformation(
    (0): WordSwapNeighboringCharacterSwap(
        (random_one):  False
      )
    (1): WordSwapRandomCharacterDeletion(
        (random_one):  False
      )
    (2): WordSwapRandomCharacterInsertion(
        (random_one):  False
      )
    (3): WordSwapQWERTY
    (4): WordSwapEmbedding(
        (max_candidates):  5
        (embedding):  WordEmbedding
      )
    )
  (constraints): 
    (0): MaxWordsPerturbed(
        (max_num_words):  4
        (compare_against_original):  True
      )
    (1): MinWordLength
    (2): StopwordModification
    (3): RepeatModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:   1%|          | 1/100 [00:04<07:12,  4.37s/it]

--------------------------------------------- Result 1 ---------------------------------------------
[[Positive (100%)]] --> [[Negative (83%)]]

lovingly photographed in the manner of a golden book sprung to life , stuart little 2 [[manages]] [[sweetness]] largely without stickiness .

lovingly photographed in the manner of a golden book sprung to life , stuart little 2 [[administering]] [[sweetnfss]] largely without stickiness .




[Succeeded / Failed / Skipped / Total] 2 / 0 / 1 / 3:   3%|▎         | 3/100 [00:08<04:33,  2.81s/it]

...

--------------------------------------------- Result 98 ---------------------------------------------
[[Positive (95%)]] --> [[Negative (79%)]]

connoisseurs of chinese film will be pleased to [[discover]] that [[tian's]] meticulous talent has not withered during his enforced [[hiatus]] .

connoisseurs of chinese film will be pleased to [[discovr]] that [[tina's]] meticulous talent has not withered during his enforced [[hitus]] .




[Succeeded / Failed / Skipped / Total] 74 / 15 / 10 / 99:  99%|█████████▉| 99/100 [07:04<00:04,  4.29s/it]

--------------------------------------------- Result 99 ---------------------------------------------
[[Positive (99%)]] --> [[Negative (76%)]]

if you can push on through the slow spots , you'll be [[rewarded]] with some fine acting .

if you can push on through the slow spots , you'll be [[rewared]] with some fine acting .




[Succeeded / Failed / Skipped / Total] 75 / 15 / 10 / 100: 100%|██████████| 100/100 [07:07<00:00,  4.28s/it]

--------------------------------------------- Result 100 ---------------------------------------------
[[Positive (96%)]] --> [[Negative (95%)]]

an unusually dry-eyed , even [[analytical]] approach to material that is generally played for maximum moisture .

an unusually dry-eyed , even [[abalytical]] approach to material that is generally played for maximum moisture .



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 75     |
| Number of failed attacks:     | 15     |
| Number of skipped attacks:    | 10     |
| Original accuracy:            | 90.0%  |
| Accuracy under attack:        | 15.0%  |
| Attack success rate:          | 83.33% |
| Average perturbed word %:     | 12.65% |
| Average num. words per input: | 18.45  |
| Avg num queries:              | 127.13 |
+-------------------------------+--------+


In [9]:
class Recipe5(Attack,ABC):
    @staticmethod
    def build(model_wrapper):
         # a combination of 4 different character-based transforms
        # ignore the first and last letter of each word, as in the paper
        transformation = CompositeTransformation(
            [
                WordSwapNeighboringCharacterSwap(
                    random_one=False, skip_first_char=True, skip_last_char=True
                ),
                WordSwapRandomCharacterDeletion(
                    random_one=False, skip_first_char=True, skip_last_char=True
                ),
                WordSwapRandomCharacterInsertion(
                    random_one=False, skip_first_char=True, skip_last_char=True
                ),
                WordSwapQWERTY(
                    random_one=False, skip_first_char=True, skip_last_char=True
                ),
                WordSwapEmbedding(max_candidates=5),
            ]
        )
        constraints = [
            MinWordLength(min_length=6),
            StopwordModification(),
            RepeatModification(),
        ]
        # untargeted attack
        goal_function = UntargetedClassification(model_wrapper)
        search_method = GreedyWordSwapWIR(wir_method="delete")
        return Attack(goal_function, constraints, transformation, search_method)

attack = Recipe5.build(model_wrapper)
attack_args = AttackArgs(num_examples=100,)
attacker = Attacker(attack, dataset,attack_args)
attack_results = attacker.attack_dataset()

textattack: Unknown if model of class <class 'transformers.models.albert.modeling_albert.AlbertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
  0%|          | 0/100 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  CompositeTransformation(
    (0): WordSwapNeighboringCharacterSwap(
        (random_one):  False
      )
    (1): WordSwapRandomCharacterDeletion(
        (random_one):  False
      )
    (2): WordSwapRandomCharacterInsertion(
        (random_one):  False
      )
    (3): WordSwapQWERTY
    (4): WordSwapEmbedding(
        (max_candidates):  5
        (embedding):  WordEmbedding
      )
    )
  (constraints): 
    (0): MinWordLength
    (1): StopwordModification
    (2): RepeatModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:   1%|          | 1/100 [00:04<07:51,  4.77s/it]

--------------------------------------------- Result 1 ---------------------------------------------
[[Positive (100%)]] --> [[Negative (83%)]]

lovingly photographed in the manner of a golden book sprung to life , stuart little 2 [[manages]] [[sweetness]] largely without stickiness .

lovingly photographed in the manner of a golden book sprung to life , stuart little 2 [[administering]] [[sweetnfss]] largely without stickiness .




[Succeeded / Failed / Skipped / Total] 2 / 0 / 1 / 3:   3%|▎         | 3/100 [00:09<05:00,  3.10s/it]

...

--------------------------------------------- Result 98 ---------------------------------------------
[[Positive (95%)]] --> [[Negative (79%)]]

connoisseurs of chinese film will be pleased to [[discover]] that [[tian's]] meticulous talent has not withered during his enforced [[hiatus]] .

connoisseurs of chinese film will be pleased to [[discovr]] that [[tina's]] meticulous talent has not withered during his enforced [[hitus]] .




[Succeeded / Failed / Skipped / Total] 83 / 6 / 10 / 99:  99%|█████████▉| 99/100 [08:14<00:04,  5.00s/it]

--------------------------------------------- Result 99 ---------------------------------------------
[[Positive (99%)]] --> [[Negative (76%)]]

if you can push on through the slow spots , you'll be [[rewarded]] with some fine acting .

if you can push on through the slow spots , you'll be [[rewared]] with some fine acting .




[Succeeded / Failed / Skipped / Total] 84 / 6 / 10 / 100: 100%|██████████| 100/100 [08:17<00:00,  4.98s/it]

--------------------------------------------- Result 100 ---------------------------------------------
[[Positive (96%)]] --> [[Negative (95%)]]

an unusually dry-eyed , even [[analytical]] approach to material that is generally played for maximum moisture .

an unusually dry-eyed , even [[abalytical]] approach to material that is generally played for maximum moisture .



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 84     |
| Number of failed attacks:     | 6      |
| Number of skipped attacks:    | 10     |
| Original accuracy:            | 90.0%  |
| Accuracy under attack:        | 6.0%   |
| Attack success rate:          | 93.33% |
| Average perturbed word %:     | 14.61% |
| Average num. words per input: | 18.45  |
| Avg num queries:              | 144.13 |
+-------------------------------+--------+


In [10]:
class Recipe5(Attack,ABC):
    @staticmethod
    def build(model_wrapper):
         # a combination of 4 different character-based transforms
        # ignore the first and last letter of each word, as in the paper
        transformation = CompositeTransformation(
            [
                WordSwapNeighboringCharacterSwap(
                    random_one=True, skip_first_char=True, skip_last_char=True
                ),
                WordSwapRandomCharacterDeletion(
                    random_one=True, skip_first_char=True, skip_last_char=True
                ),
                WordSwapRandomCharacterInsertion(
                    random_one=True, skip_first_char=True, skip_last_char=True
                ),
                WordSwapQWERTY(
                    random_one=False, skip_first_char=True, skip_last_char=True
                ),
                WordSwapEmbedding(max_candidates=5),
            ]
        )
        constraints = [
            MinWordLength(min_length=6),
            StopwordModification(),
            MaxWordsPerturbed(max_num_words=4),
            RepeatModification(),
        ]
        # untargeted attack
        goal_function = UntargetedClassification(model_wrapper)
        search_method = GreedyWordSwapWIR(wir_method="delete")
        return Attack(goal_function, constraints, transformation, search_method)

attack = Recipe5.build(model_wrapper)
attack_args = AttackArgs(num_examples=100,)
attacker = Attacker(attack, dataset,attack_args)
attack_results = attacker.attack_dataset()

textattack: Unknown if model of class <class 'transformers.models.albert.modeling_albert.AlbertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
  0%|          | 0/100 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  CompositeTransformation(
    (0): WordSwapNeighboringCharacterSwap(
        (random_one):  True
      )
    (1): WordSwapRandomCharacterDeletion(
        (random_one):  True
      )
    (2): WordSwapRandomCharacterInsertion(
        (random_one):  True
      )
    (3): WordSwapQWERTY
    (4): WordSwapEmbedding(
        (max_candidates):  5
        (embedding):  WordEmbedding
      )
    )
  (constraints): 
    (0): MaxWordsPerturbed(
        (max_num_words):  4
        (compare_against_original):  True
      )
    (1): MinWordLength
    (2): StopwordModification
    (3): RepeatModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:   1%|          | 1/100 [00:03<06:17,  3.81s/it]

--------------------------------------------- Result 1 ---------------------------------------------
[[Positive (100%)]] --> [[Negative (83%)]]

lovingly photographed in the manner of a golden book sprung to life , stuart little 2 [[manages]] [[sweetness]] largely without stickiness .

lovingly photographed in the manner of a golden book sprung to life , stuart little 2 [[administering]] [[sweetnfss]] largely without stickiness .




[Succeeded / Failed / Skipped / Total] 2 / 0 / 1 / 3:   3%|▎         | 3/100 [00:07<03:50,  2.38s/it]

...

--------------------------------------------- Result 98 ---------------------------------------------
[[Positive (95%)]] --> [[Negative (66%)]]

connoisseurs of chinese film will be pleased to [[discover]] that [[tian's]] meticulous talent has not withered during his enforced [[hiatus]] .

connoisseurs of chinese film will be pleased to [[discocer]] that [[tizn's]] meticulous talent has not withered during his enforced [[hitus]] .




[Succeeded / Failed / Skipped / Total] 71 / 18 / 10 / 99:  99%|█████████▉| 99/100 [05:54<00:03,  3.59s/it]

--------------------------------------------- Result 99 ---------------------------------------------
[[Positive (99%)]] --> [[Negative (69%)]]

if you can push on through the slow spots , you'll be [[rewarded]] with some fine acting .

if you can push on through the slow spots , you'll be [[rewarfed]] with some fine acting .




[Succeeded / Failed / Skipped / Total] 72 / 18 / 10 / 100: 100%|██████████| 100/100 [05:57<00:00,  3.57s/it]

--------------------------------------------- Result 100 ---------------------------------------------
[[Positive (96%)]] --> [[Negative (95%)]]

an unusually dry-eyed , even [[analytical]] approach to material that is generally played for maximum moisture .

an unusually dry-eyed , even [[abalytical]] approach to material that is generally played for maximum moisture .



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 72     |
| Number of failed attacks:     | 18     |
| Number of skipped attacks:    | 10     |
| Original accuracy:            | 90.0%  |
| Accuracy under attack:        | 18.0%  |
| Attack success rate:          | 80.0%  |
| Average perturbed word %:     | 13.36% |
| Average num. words per input: | 18.45  |
| Avg num queries:              | 103.39 |
+-------------------------------+--------+


In [11]:
class Recipe6(Attack,ABC):
    @staticmethod
    def build(model_wrapper):
         # a combination of 4 different character-based transforms
        # ignore the first and last letter of each word, as in the paper
        transformation = CompositeTransformation(
            [
                WordSwapNeighboringCharacterSwap(
                    random_one=True, skip_first_char=True, skip_last_char=True
                ),
                WordSwapRandomCharacterDeletion(
                    random_one=True, skip_first_char=True, skip_last_char=True
                ),
                WordSwapRandomCharacterInsertion(
                    random_one=True, skip_first_char=True, skip_last_char=True
                ),
                WordSwapQWERTY(
                    random_one=False, skip_first_char=True, skip_last_char=True
                ),
                WordSwapEmbedding(max_candidates=5),
            ]
        )
        constraints = [
            MinWordLength(min_length=2),
            #StopwordModification(),
            MaxWordsPerturbed(max_num_words=4),
            RepeatModification(),
        ]
        # untargeted attack
        goal_function = UntargetedClassification(model_wrapper)
        search_method = GreedyWordSwapWIR(wir_method="delete")
        return Attack(goal_function, constraints, transformation, search_method)

attack = Recipe6.build(model_wrapper)
attack_args = AttackArgs(num_examples=100,)
attacker = Attacker(attack, dataset,attack_args)
attack_results = attacker.attack_dataset()

textattack: Unknown if model of class <class 'transformers.models.albert.modeling_albert.AlbertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
  0%|          | 0/100 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  CompositeTransformation(
    (0): WordSwapNeighboringCharacterSwap(
        (random_one):  True
      )
    (1): WordSwapRandomCharacterDeletion(
        (random_one):  True
      )
    (2): WordSwapRandomCharacterInsertion(
        (random_one):  True
      )
    (3): WordSwapQWERTY
    (4): WordSwapEmbedding(
        (max_candidates):  5
        (embedding):  WordEmbedding
      )
    )
  (constraints): 
    (0): MaxWordsPerturbed(
        (max_num_words):  4
        (compare_against_original):  True
      )
    (1): MinWordLength
    (2): RepeatModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:   1%|          | 1/100 [00:03<06:15,  3.80s/it]

--------------------------------------------- Result 1 ---------------------------------------------
[[Positive (100%)]] --> [[Negative (83%)]]

lovingly photographed in the manner of a golden book sprung to life , stuart little 2 [[manages]] [[sweetness]] largely without stickiness .

lovingly photographed in the manner of a golden book sprung to life , stuart little 2 [[administering]] [[sweetnfss]] largely without stickiness .




[Succeeded / Failed / Skipped / Total] 2 / 0 / 1 / 3:   3%|▎         | 3/100 [00:07<03:49,  2.37s/it]

...

--------------------------------------------- Result 98 ---------------------------------------------
[[Positive (95%)]] --> [[Negative (85%)]]

connoisseurs of chinese film will be pleased to discover that tian's meticulous talent has [[not]] withered during his enforced hiatus .

connoisseurs of chinese film will be pleased to discover that tian's meticulous talent has [[nah]] withered during his enforced hiatus .




[Succeeded / Failed / Skipped / Total] 75 / 14 / 10 / 99:  99%|█████████▉| 99/100 [04:30<00:02,  2.73s/it]

--------------------------------------------- Result 99 ---------------------------------------------
[[Positive (99%)]] --> [[Negative (90%)]]

if you can push on through the slow spots , you'll be rewarded with some [[fine]] acting .

if you can push on through the slow spots , you'll be rewarded with some [[fjne]] acting .




[Succeeded / Failed / Skipped / Total] 76 / 14 / 10 / 100: 100%|██████████| 100/100 [04:32<00:00,  2.72s/it]

--------------------------------------------- Result 100 ---------------------------------------------
[[Positive (96%)]] --> [[Negative (95%)]]

an unusually dry-eyed , even [[analytical]] approach to material that is generally played for maximum moisture .

an unusually dry-eyed , even [[abalytical]] approach to material that is generally played for maximum moisture .



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 76     |
| Number of failed attacks:     | 14     |
| Number of skipped attacks:    | 10     |
| Original accuracy:            | 90.0%  |
| Accuracy under attack:        | 14.0%  |
| Attack success rate:          | 84.44% |
| Average perturbed word %:     | 13.08% |
| Average num. words per input: | 18.45  |
| Avg num queries:              | 78.56  |
+-------------------------------+--------+


In [7]:
class Recipe7(Attack,ABC):
    @staticmethod
    def build(model_wrapper):
         # a combination of 4 different character-based transforms
        # ignore the first and last letter of each word, as in the paper
        transformation = CompositeTransformation(
            [
                
                WordSwapRandomCharacterDeletion(
                    random_one=True, skip_first_char=True, skip_last_char=True
                ),
                WordSwapRandomCharacterInsertion(
                    random_one=True, skip_first_char=True, skip_last_char=True
                ),
                WordSwapQWERTY(
                    random_one=False, skip_first_char=True, skip_last_char=True
                ),
                WordSwapEmbedding(max_candidates=5),
            ]
        )
        constraints = [
            MinWordLength(min_length=6),
            StopwordModification(),
            MaxWordsPerturbed(max_num_words=4),
            RepeatModification(),
        ]
        # untargeted attack
        goal_function = UntargetedClassification(model_wrapper)
        search_method = GreedyWordSwapWIR(wir_method="delete")
        return Attack(goal_function, constraints, transformation, search_method)

attack = Recipe7.build(model_wrapper)
attack_args = AttackArgs(num_examples=100,)
attacker = Attacker(attack, dataset,attack_args)
attack_results = attacker.attack_dataset()

textattack: Unknown if model of class <class 'transformers.models.albert.modeling_albert.AlbertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
  0%|          | 0/100 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  CompositeTransformation(
    (0): WordSwapRandomCharacterDeletion(
        (random_one):  True
      )
    (1): WordSwapRandomCharacterInsertion(
        (random_one):  True
      )
    (2): WordSwapQWERTY
    (3): WordSwapEmbedding(
        (max_candidates):  5
        (embedding):  WordEmbedding
      )
    )
  (constraints): 
    (0): MaxWordsPerturbed(
        (max_num_words):  4
        (compare_against_original):  True
      )
    (1): MinWordLength
    (2): StopwordModification
    (3): RepeatModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:   1%|          | 1/100 [00:02<03:19,  2.01s/it]

--------------------------------------------- Result 1 ---------------------------------------------
[[Positive (100%)]] --> [[Negative (83%)]]

lovingly photographed in the manner of a golden book sprung to life , stuart little 2 [[manages]] [[sweetness]] largely without stickiness .

lovingly photographed in the manner of a golden book sprung to life , stuart little 2 [[administering]] [[sweetnfss]] largely without stickiness .




[Succeeded / Failed / Skipped / Total] 2 / 0 / 1 / 3:   3%|▎         | 3/100 [00:03<02:01,  1.25s/it]

...

--------------------------------------------- Result 98 ---------------------------------------------
[[Positive (95%)]] --> [[Negative (52%)]]

connoisseurs of chinese film will be pleased to [[discover]] that tian's meticulous talent has not withered during his enforced [[hiatus]] .

connoisseurs of chinese film will be pleased to [[discovier]] that tian's meticulous talent has not withered during his enforced [[hiaXtus]] .




[Succeeded / Failed / Skipped / Total] 72 / 17 / 10 / 99:  99%|█████████▉| 99/100 [03:02<00:01,  1.85s/it]

--------------------------------------------- Result 99 ---------------------------------------------
[[Positive (99%)]] --> [[Negative (69%)]]

if you can push on through the slow spots , you'll be [[rewarded]] with some fine acting .

if you can push on through the slow spots , you'll be [[rewarfed]] with some fine acting .




[Succeeded / Failed / Skipped / Total] 73 / 17 / 10 / 100: 100%|██████████| 100/100 [03:04<00:00,  1.84s/it]

--------------------------------------------- Result 100 ---------------------------------------------
[[Positive (96%)]] --> [[Negative (95%)]]

an unusually dry-eyed , even [[analytical]] approach to material that is generally played for maximum moisture .

an unusually dry-eyed , even [[abalytical]] approach to material that is generally played for maximum moisture .



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 73     |
| Number of failed attacks:     | 17     |
| Number of skipped attacks:    | 10     |
| Original accuracy:            | 90.0%  |
| Accuracy under attack:        | 17.0%  |
| Attack success rate:          | 81.11% |
| Average perturbed word %:     | 13.07% |
| Average num. words per input: | 18.45  |
| Avg num queries:              | 99.33  |
+-------------------------------+--------+


In [8]:
class Recipe8(Attack,ABC):
    @staticmethod
    def build(model_wrapper):
         # a combination of 4 different character-based transforms
        # ignore the first and last letter of each word, as in the paper
        transformation = CompositeTransformation(
            [
                WordSwapNeighboringCharacterSwap(
                    random_one=True, skip_first_char=True, skip_last_char=True
                ),
               
                WordSwapRandomCharacterInsertion(
                    random_one=True, skip_first_char=True, skip_last_char=True
                ),
                WordSwapQWERTY(
                    random_one=False, skip_first_char=True, skip_last_char=True
                ),
                WordSwapEmbedding(max_candidates=5),
            ]
        )
        constraints = [
            MinWordLength(min_length=6),
            StopwordModification(),
            MaxWordsPerturbed(max_num_words=4),
            RepeatModification(),
        ]
        # untargeted attack
        goal_function = UntargetedClassification(model_wrapper)
        search_method = GreedyWordSwapWIR(wir_method="delete")
        return Attack(goal_function, constraints, transformation, search_method)

attack = Recipe8.build(model_wrapper)
attack_args = AttackArgs(num_examples=100,)
attacker = Attacker(attack, dataset,attack_args)
attack_results = attacker.attack_dataset()

textattack: Unknown if model of class <class 'transformers.models.albert.modeling_albert.AlbertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
  0%|          | 0/100 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  CompositeTransformation(
    (0): WordSwapNeighboringCharacterSwap(
        (random_one):  True
      )
    (1): WordSwapRandomCharacterInsertion(
        (random_one):  True
      )
    (2): WordSwapQWERTY
    (3): WordSwapEmbedding(
        (max_candidates):  5
        (embedding):  WordEmbedding
      )
    )
  (constraints): 
    (0): MaxWordsPerturbed(
        (max_num_words):  4
        (compare_against_original):  True
      )
    (1): MinWordLength
    (2): StopwordModification
    (3): RepeatModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:   1%|          | 1/100 [00:02<03:19,  2.01s/it]

--------------------------------------------- Result 1 ---------------------------------------------
[[Positive (100%)]] --> [[Negative (83%)]]

lovingly photographed in the manner of a golden book sprung to life , stuart little 2 [[manages]] [[sweetness]] largely without stickiness .

lovingly photographed in the manner of a golden book sprung to life , stuart little 2 [[administering]] [[sweetnfss]] largely without stickiness .




[Succeeded / Failed / Skipped / Total] 2 / 0 / 1 / 3:   3%|▎         | 3/100 [00:04<02:38,  1.64s/it]

...

--------------------------------------------- Result 98 ---------------------------------------------
[[Positive (95%)]] --> [[Negative (65%)]]

connoisseurs of chinese film will be pleased to [[discover]] that [[tian's]] [[meticulous]] talent has not withered during his enforced [[hiatus]] .

connoisseurs of chinese film will be pleased to [[discocer]] that [[toan's]] [[msticulous]] talent has not withered during his enforced [[hiatis]] .




[Succeeded / Failed / Skipped / Total] 72 / 17 / 10 / 99:  99%|█████████▉| 99/100 [03:06<00:01,  1.88s/it]

--------------------------------------------- Result 99 ---------------------------------------------
[[Positive (99%)]] --> [[Negative (69%)]]

if you can push on through the slow spots , you'll be [[rewarded]] with some fine acting .

if you can push on through the slow spots , you'll be [[rewarfed]] with some fine acting .




[Succeeded / Failed / Skipped / Total] 73 / 17 / 10 / 100: 100%|██████████| 100/100 [03:07<00:00,  1.88s/it]

--------------------------------------------- Result 100 ---------------------------------------------
[[Positive (96%)]] --> [[Negative (95%)]]

an unusually dry-eyed , even [[analytical]] approach to material that is generally played for maximum moisture .

an unusually dry-eyed , even [[abalytical]] approach to material that is generally played for maximum moisture .



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 73     |
| Number of failed attacks:     | 17     |
| Number of skipped attacks:    | 10     |
| Original accuracy:            | 90.0%  |
| Accuracy under attack:        | 17.0%  |
| Attack success rate:          | 81.11% |
| Average perturbed word %:     | 13.57% |
| Average num. words per input: | 18.45  |
| Avg num queries:              | 101.2  |
+-------------------------------+--------+


In [9]:
class Recipe9(Attack,ABC):
    @staticmethod
    def build(model_wrapper):
         # a combination of 4 different character-based transforms
        # ignore the first and last letter of each word, as in the paper
        transformation = CompositeTransformation(
            [
                WordSwapNeighboringCharacterSwap(
                    random_one=True, skip_first_char=True, skip_last_char=True
                ),
                WordSwapRandomCharacterDeletion(
                    random_one=True, skip_first_char=True, skip_last_char=True
                ),
              
                WordSwapQWERTY(
                    random_one=False, skip_first_char=True, skip_last_char=True
                ),
                WordSwapEmbedding(max_candidates=5),
            ]
        )
        constraints = [
            MinWordLength(min_length=6),
            StopwordModification(),
            MaxWordsPerturbed(max_num_words=4),
            RepeatModification(),
        ]
        # untargeted attack
        goal_function = UntargetedClassification(model_wrapper)
        search_method = GreedyWordSwapWIR(wir_method="delete")
        return Attack(goal_function, constraints, transformation, search_method)

attack = Recipe9.build(model_wrapper)
attack_args = AttackArgs(num_examples=100,)
attacker = Attacker(attack, dataset,attack_args)
attack_results = attacker.attack_dataset()

textattack: Unknown if model of class <class 'transformers.models.albert.modeling_albert.AlbertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
  0%|          | 0/100 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  CompositeTransformation(
    (0): WordSwapNeighboringCharacterSwap(
        (random_one):  True
      )
    (1): WordSwapRandomCharacterDeletion(
        (random_one):  True
      )
    (2): WordSwapQWERTY
    (3): WordSwapEmbedding(
        (max_candidates):  5
        (embedding):  WordEmbedding
      )
    )
  (constraints): 
    (0): MaxWordsPerturbed(
        (max_num_words):  4
        (compare_against_original):  True
      )
    (1): MinWordLength
    (2): StopwordModification
    (3): RepeatModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:   1%|          | 1/100 [00:02<03:19,  2.01s/it]

--------------------------------------------- Result 1 ---------------------------------------------
[[Positive (100%)]] --> [[Negative (83%)]]

lovingly photographed in the manner of a golden book sprung to life , stuart little 2 [[manages]] [[sweetness]] largely without stickiness .

lovingly photographed in the manner of a golden book sprung to life , stuart little 2 [[administering]] [[sweetnfss]] largely without stickiness .




[Succeeded / Failed / Skipped / Total] 2 / 0 / 1 / 3:   3%|▎         | 3/100 [00:04<02:38,  1.63s/it]

...

--------------------------------------------- Result 98 ---------------------------------------------
[[Positive (95%)]] --> [[Negative (73%)]]

connoisseurs of chinese film will be pleased to [[discover]] that [[tian's]] meticulous talent has not withered during his enforced [[hiatus]] .

connoisseurs of chinese film will be pleased to [[discocer]] that [[tina's]] meticulous talent has not withered during his enforced [[hatus]] .




[Succeeded / Failed / Skipped / Total] 73 / 16 / 10 / 99:  99%|█████████▉| 99/100 [03:06<00:01,  1.89s/it]

--------------------------------------------- Result 99 ---------------------------------------------
[[Positive (99%)]] --> [[Negative (76%)]]

if you can push on through the slow spots , you'll be [[rewarded]] with some fine acting .

if you can push on through the slow spots , you'll be [[rewared]] with some fine acting .




[Succeeded / Failed / Skipped / Total] 74 / 16 / 10 / 100: 100%|██████████| 100/100 [03:08<00:00,  1.88s/it]

--------------------------------------------- Result 100 ---------------------------------------------
[[Positive (96%)]] --> [[Negative (95%)]]

an unusually dry-eyed , even [[analytical]] approach to material that is generally played for maximum moisture .

an unusually dry-eyed , even [[abalytical]] approach to material that is generally played for maximum moisture .



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 74     |
| Number of failed attacks:     | 16     |
| Number of skipped attacks:    | 10     |
| Original accuracy:            | 90.0%  |
| Accuracy under attack:        | 16.0%  |
| Attack success rate:          | 82.22% |
| Average perturbed word %:     | 13.97% |
| Average num. words per input: | 18.45  |
| Avg num queries:              | 101.54 |
+-------------------------------+--------+


In [10]:
class Recipe10(Attack,ABC):
    @staticmethod
    def build(model_wrapper):
         # a combination of 4 different character-based transforms
        # ignore the first and last letter of each word, as in the paper
        transformation = CompositeTransformation(
            [
                WordSwapNeighboringCharacterSwap(
                    random_one=True, skip_first_char=True, skip_last_char=True
                ),
                WordSwapRandomCharacterDeletion(
                    random_one=True, skip_first_char=True, skip_last_char=True
                ),
                WordSwapRandomCharacterInsertion(
                    random_one=True, skip_first_char=True, skip_last_char=True
                ),
              
                WordSwapEmbedding(max_candidates=5),
            ]
        )
        constraints = [
            MinWordLength(min_length=6),
            StopwordModification(),
            MaxWordsPerturbed(max_num_words=4),
            RepeatModification(),
        ]
        # untargeted attack
        goal_function = UntargetedClassification(model_wrapper)
        search_method = GreedyWordSwapWIR(wir_method="delete")
        return Attack(goal_function, constraints, transformation, search_method)

attack = Recipe10.build(model_wrapper)
attack_args = AttackArgs(num_examples=100,)
attacker = Attacker(attack, dataset,attack_args)
attack_results = attacker.attack_dataset()

textattack: Unknown if model of class <class 'transformers.models.albert.modeling_albert.AlbertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
  0%|          | 0/100 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  CompositeTransformation(
    (0): WordSwapNeighboringCharacterSwap(
        (random_one):  True
      )
    (1): WordSwapRandomCharacterDeletion(
        (random_one):  True
      )
    (2): WordSwapRandomCharacterInsertion(
        (random_one):  True
      )
    (3): WordSwapEmbedding(
        (max_candidates):  5
        (embedding):  WordEmbedding
      )
    )
  (constraints): 
    (0): MaxWordsPerturbed(
        (max_num_words):  4
        (compare_against_original):  True
      )
    (1): MinWordLength
    (2): StopwordModification
    (3): RepeatModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:   1%|          | 1/100 [00:00<01:20,  1.23it/s]

--------------------------------------------- Result 1 ---------------------------------------------
[[Positive (100%)]] --> [[Negative (82%)]]

lovingly photographed in the manner of a golden book sprung to life , stuart little 2 [[manages]] [[sweetness]] largely without stickiness .

lovingly photographed in the manner of a golden book sprung to life , stuart little 2 [[administering]] [[sweentess]] largely without stickiness .




[Succeeded / Failed / Skipped / Total] 2 / 0 / 1 / 3:   3%|▎         | 3/100 [00:01<00:42,  2.30it/s]

...

--------------------------------------------- Result 98 ---------------------------------------------
[[Positive (95%)]] --> [[[FAILED]]]

connoisseurs of chinese film will be pleased to discover that tian's meticulous talent has not withered during his enforced hiatus .




[Succeeded / Failed / Skipped / Total] 54 / 35 / 10 / 99:  99%|█████████▉| 99/100 [01:18<00:00,  1.26it/s]

--------------------------------------------- Result 99 ---------------------------------------------
[[Positive (99%)]] --> [[Negative (60%)]]

if you can push on through the slow spots , you'll be [[rewarded]] with some fine acting .

if you can push on through the slow spots , you'll be [[compensated]] with some fine acting .




[Succeeded / Failed / Skipped / Total] 55 / 35 / 10 / 100: 100%|██████████| 100/100 [01:18<00:00,  1.27it/s]

--------------------------------------------- Result 100 ---------------------------------------------
[[Positive (96%)]] --> [[Negative (84%)]]

an unusually dry-eyed , even [[analytical]] approach to material that is generally played for maximum moisture .

an unusually dry-eyed , even [[anallytical]] approach to material that is generally played for maximum moisture .



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 55     |
| Number of failed attacks:     | 35     |
| Number of skipped attacks:    | 10     |
| Original accuracy:            | 90.0%  |
| Accuracy under attack:        | 35.0%  |
| Attack success rate:          | 61.11% |
| Average perturbed word %:     | 15.2%  |
| Average num. words per input: | 18.45  |
| Avg num queries:              | 40.52  |
+-------------------------------+--------+


In [11]:
class Recipe11(Attack,ABC):
    @staticmethod
    def build(model_wrapper):
         # a combination of 4 different character-based transforms
        # ignore the first and last letter of each word, as in the paper
        transformation = CompositeTransformation(
            [
                WordSwapNeighboringCharacterSwap(
                    random_one=True, skip_first_char=True, skip_last_char=True
                ),
                WordSwapRandomCharacterDeletion(
                    random_one=True, skip_first_char=True, skip_last_char=True
                ),
                WordSwapRandomCharacterInsertion(
                    random_one=True, skip_first_char=True, skip_last_char=True
                ),
                
            ]
        )
        constraints = [
            MinWordLength(min_length=6),
            StopwordModification(),
            MaxWordsPerturbed(max_num_words=4),
            RepeatModification(),
        ]
        # untargeted attack
        goal_function = UntargetedClassification(model_wrapper)
        search_method = GreedyWordSwapWIR(wir_method="delete")
        return Attack(goal_function, constraints, transformation, search_method)

attack = Recipe11.build(model_wrapper)
attack_args = AttackArgs(num_examples=100,)
attacker = Attacker(attack, dataset,attack_args)
attack_results = attacker.attack_dataset()

textattack: Unknown if model of class <class 'transformers.models.albert.modeling_albert.AlbertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
  0%|          | 0/100 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  CompositeTransformation(
    (0): WordSwapNeighboringCharacterSwap(
        (random_one):  True
      )
    (1): WordSwapRandomCharacterDeletion(
        (random_one):  True
      )
    (2): WordSwapRandomCharacterInsertion(
        (random_one):  True
      )
    )
  (constraints): 
    (0): MaxWordsPerturbed(
        (max_num_words):  4
        (compare_against_original):  True
      )
    (1): MinWordLength
    (2): StopwordModification
    (3): RepeatModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:   1%|          | 1/100 [00:00<00:59,  1.67it/s]

--------------------------------------------- Result 1 ---------------------------------------------
[[Positive (100%)]] --> [[Negative (85%)]]

lovingly photographed in the manner of a golden book sprung to life , stuart little 2 [[manages]] [[sweetness]] largely without stickiness .

lovingly photographed in the manner of a golden book sprung to life , stuart little 2 [[managces]] [[swetness]] largely without stickiness .




[Succeeded / Failed / Skipped / Total] 2 / 0 / 1 / 3:   3%|▎         | 3/100 [00:00<00:28,  3.39it/s]

...

--------------------------------------------- Result 98 ---------------------------------------------
[[Positive (95%)]] --> [[[FAILED]]]

connoisseurs of chinese film will be pleased to discover that tian's meticulous talent has not withered during his enforced hiatus .




[Succeeded / Failed / Skipped / Total] 44 / 45 / 10 / 99:  99%|█████████▉| 99/100 [00:56<00:00,  1.75it/s]

--------------------------------------------- Result 99 ---------------------------------------------
[[Positive (99%)]] --> [[Negative (76%)]]

if you can push on through the slow spots , you'll be [[rewarded]] with some fine acting .

if you can push on through the slow spots , you'll be [[rewarbded]] with some fine acting .




[Succeeded / Failed / Skipped / Total] 45 / 45 / 10 / 100: 100%|██████████| 100/100 [00:57<00:00,  1.75it/s]

--------------------------------------------- Result 100 ---------------------------------------------
[[Positive (96%)]] --> [[Negative (79%)]]

an [[unusually]] dry-eyed , even analytical [[approach]] to material that is generally played for maximum moisture .

an [[unusully]] dry-eyed , even analytical [[apprIoach]] to material that is generally played for maximum moisture .



+-------------------------------+-------+
| Attack Results                |       |
+-------------------------------+-------+
| Number of successful attacks: | 45    |
| Number of failed attacks:     | 45    |
| Number of skipped attacks:    | 10    |
| Original accuracy:            | 90.0% |
| Accuracy under attack:        | 45.0% |
| Attack success rate:          | 50.0% |
| Average perturbed word %:     | 15.4% |
| Average num. words per input: | 18.45 |
| Avg num queries:              | 28.94 |
+-------------------------------+-------+


In [12]:
class Recipe12(Attack,ABC):
    @staticmethod
    def build(model_wrapper):
         # a combination of 4 different character-based transforms
        # ignore the first and last letter of each word, as in the paper
        transformation = CompositeTransformation(
            [
                WordSwapNeighboringCharacterSwap(
                    random_one=True, skip_first_char=True, skip_last_char=True
                ),
                WordSwapRandomCharacterDeletion(
                    random_one=True, skip_first_char=True, skip_last_char=True
                ),
                WordSwapRandomCharacterInsertion(
                    random_one=True, skip_first_char=True, skip_last_char=True
                ),
                WordSwapQWERTY(
                    random_one=False, skip_first_char=True, skip_last_char=True
                ),
                WordSwapEmbedding(max_candidates=5),
            ]
        )
        constraints = [
            MinWordLength(min_length=6),
            StopwordModification(),
            MaxWordsPerturbed(max_num_words=4),
            RepeatModification(),
            MaxWordIndexModification(5) 
        ]
        # untargeted attack
        goal_function = UntargetedClassification(model_wrapper)
        search_method = GreedyWordSwapWIR(wir_method="delete")
        return Attack(goal_function, constraints, transformation, search_method)

attack = Recipe12.build(model_wrapper)
attack_args = AttackArgs(num_examples=100,)
attacker = Attacker(attack, dataset,attack_args)
attack_results = attacker.attack_dataset()

textattack: Unknown if model of class <class 'transformers.models.albert.modeling_albert.AlbertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
  0%|          | 0/100 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  CompositeTransformation(
    (0): WordSwapNeighboringCharacterSwap(
        (random_one):  True
      )
    (1): WordSwapRandomCharacterDeletion(
        (random_one):  True
      )
    (2): WordSwapRandomCharacterInsertion(
        (random_one):  True
      )
    (3): WordSwapQWERTY
    (4): WordSwapEmbedding(
        (max_candidates):  5
        (embedding):  WordEmbedding
      )
    )
  (constraints): 
    (0): MaxWordsPerturbed(
        (max_num_words):  4
        (compare_against_original):  True
      )
    (1): MinWordLength
    (2): StopwordModification
    (3): RepeatModification
    (4): MaxWordIndexModification(
        (max_length):  5
      )
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 0 / 1 / 0 / 1:   1%|          | 1/100 [00:02<04:47,  2.90s/it]

--------------------------------------------- Result 1 ---------------------------------------------
[[Positive (100%)]] --> [[[FAILED]]]

lovingly photographed in the manner of a golden book sprung to life , stuart little 2 manages sweetness largely without stickiness .




[Succeeded / Failed / Skipped / Total] 1 / 1 / 1 / 3:   3%|▎         | 3/100 [00:05<03:09,  1.95s/it]

...

--------------------------------------------- Result 98 ---------------------------------------------
[[Positive (95%)]] --> [[[FAILED]]]

connoisseurs of chinese film will be pleased to discover that tian's meticulous talent has not withered during his enforced hiatus .




[Succeeded / Failed / Skipped / Total] 36 / 53 / 10 / 99:  99%|█████████▉| 99/100 [02:17<00:01,  1.39s/it]

--------------------------------------------- Result 99 ---------------------------------------------
[[Positive (99%)]] --> [[[FAILED]]]

if you can push on through the slow spots , you'll be rewarded with some fine acting .




[Succeeded / Failed / Skipped / Total] 37 / 53 / 10 / 100: 100%|██████████| 100/100 [02:18<00:00,  1.39s/it]

--------------------------------------------- Result 100 ---------------------------------------------
[[Positive (96%)]] --> [[Negative (95%)]]

an unusually dry-eyed , even [[analytical]] approach to material that is generally played for maximum moisture .

an unusually dry-eyed , even [[abalytical]] approach to material that is generally played for maximum moisture .



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 37     |
| Number of failed attacks:     | 53     |
| Number of skipped attacks:    | 10     |
| Original accuracy:            | 90.0%  |
| Accuracy under attack:        | 53.0%  |
| Attack success rate:          | 41.11% |
| Average perturbed word %:     | 13.9%  |
| Average num. words per input: | 18.45  |
| Avg num queries:              | 74.87  |
+-------------------------------+--------+
